In [ ]:
import xarray as xr
import zarr
from zarr.codecs import BloscCodec
import glob
from swed_17.nb_helpers import start_cluster

In [ ]:
cluster = start_cluster(n_workers=12, memory_limit="8GB", local=False)

In [ ]:
BASE_DIR = "/perc10/data/cbrfcSnowModel/"
SNOBAL_DIR = BASE_DIR + "isnobal"

WATER_YEAR = "2021"
BASIN = "colkrem"

In [ ]:
snow = sorted(glob.glob(SNOBAL_DIR + f"/wy{WATER_YEAR}/{BASIN}/run*/snow.nc"))
em = sorted(glob.glob(SNOBAL_DIR + f"/wy{WATER_YEAR}/{BASIN}/run*/em.nc"))

OUTPUT_ZARR_PATH = SNOBAL_DIR + f"/isnobal/wy{WATER_YEAR}_{BASIN}.zarr"

In [ ]:
ds = xr.open_mfdataset(
    snow + em,
    combine='by_coords',
    data_vars="all",
    compat="no_conflicts",
    parallel=True
)

In [ ]:
compressor = BloscCodec(cname="zlib", clevel=4, shuffle=zarr.codecs.BloscShuffle.bitshuffle)

encoding = {}
for var_name in ds.data_vars:
    encoding[var_name] = {
        'compressors': compressor,
    }
for coord_name in ds.coords:
    encoding[coord_name] = {'compressors': compressor}

In [ ]:
ds.to_zarr(
    OUTPUT_ZARR_PATH,
    mode='w',
    encoding=encoding
)

In [ ]:
cluster.shutdown()